<h1><center></center></h1>
<h1><center>DSAI 308</center></h1>
<h1><center>Lab Exam 2 (Siamese & Autoencoders)</center></h1>
<h2><center>Model A</center></h2>
<h2><center>Exam Time: 60 minutes</center></h2>
<h2><center>Turn on GPU</center></h2>


**Exam Instructions**
- You are allowed to open TensorFlow & PyTorch documentation, no other links or colabs. Any way of cheating will be directed to **ZERO GRADE**.
- Close all the tabs in your browser and keep only the notebook tab and documentation tab open.
- Close all the windows on your operating system except for the browser window.
- keep your cell phone in your pocket or on your bag and make it on silent mode.

---



---



***References***

1. [TensorFlow Documentation](https://www.tensorflow.org/api_docs/python/tf/all_symbols)
2. [PyTorch Documentation](https://pytorch.org/docs/stable/index.html)
3. [Keras Documentation](https://keras.io)

## **Task 1 - Siamese Network for Similarity Learning**  

Implement a **Siamese Network** to learn similarity between images from the **CIFAR-10 dataset**.

### Instructions:

1. **Load and Preprocess the CIFAR-10 Dataset:**
   - Load the **CIFAR-10** dataset containing images of handwritten digits.
   - Normalize the pixel values of images to the range `[0, 1]`.
   - Create **pairs of images**:  
     - Positive pairs: Images of the same digit.  
     - Negative pairs: Images of different digits.

2. **Build the Siamese Network:**
   - **Base Network:**  
     - Input: `(28, 28, 1)` image.
     - Add **Conv2D** layers to extract features from the input images.
     - Use **MaxPooling2D** to reduce spatial dimensions.
     - Add a **Dense layer** to output a feature vector representing the input image.

   - **Siamese Architecture:**  
     - Use the base network to extract features from two input images.
     - Compute the **L1 distance** between the feature vectors.
     - Add a **Dense layer** with a sigmoid activation to predict similarity (1 = similar, 0 = dissimilar).

3. **Train the Siamese Network:**
   - Use the appropriate loss and optimizer.
   - Train for at least **10 epochs** and do not forget the batch size, try to get +65% accuracy

4. **Evaluate the Siamese Network:**
   - Report the **test accuracy**



In [86]:
# Task 1: Siamese Network on CIFAR-10

import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.datasets import cifar10
import numpy as np

# Create pairs for Siamese Network
def make_pairs(images, labels):
    pairImages = []
    pairLabels = []
    numClasses = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(numClasses)]

    for idxA in range(len(images)):
        currentImage = images[idxA]
        label = labels[idxA]

        idxB = np.random.choice(idx[label])
        posImage = images[idxB]

        pairImages.append([currentImage, posImage])
        pairLabels.append([1])

        negIdx = np.where(labels != label)[0]
        negImage = images[np.random.choice(negIdx)]

        pairImages.append([currentImage, negImage])
        pairLabels.append([0])

    return np.array(pairImages), np.array(pairLabels)

def euclidean_distance(vects):
    x, y = vects
    return tf.math.sqrt(tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True))


# Load CIFAR-10 dataset
(trainX, trainY), (testX, testY) = cifar10.load_data()

trainX, trainY, testX, testY = trainX[:20000], trainY[:20000], testX[:20000], testY[:20000]

trainY = trainY.flatten()
testY = testY.flatten()
x_train_val = trainX.astype("float32")
testX = testX.astype("float32")

In [87]:
# [0.5 mark] Preprocess input & apply pairs
pairs_train, labels_train = make_pairs(trainX, trainY)

# make validation pairs
pairs_val, labels_val = make_pairs(testX, testY)

# make test pairs
pairs_test, labels_test = make_pairs(testX, testY)

In [88]:
trainX.shape

(20000, 32, 32, 3)

In [89]:
import random
import numpy as np
import keras
from keras import ops
import matplotlib.pyplot as plt

In [90]:
# [1.5 mark]

# TODO: Define a CNN model
# The model should contain Conv2D, relu, MaxPool2D, and Fully Connected layers
# Define the CNN model for feature extraction
# Build the Siamese Network
input = keras.layers.Input((32, 32, 3))
x = keras.layers.BatchNormalization()(input)
x = keras.layers.Conv2D(4, (5, 5), activation="relu")(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Conv2D(16, (5, 5), activation="relu")(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

x = keras.layers.Flatten()(x)

x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(10, activation="sigmoid")(x)
embedding_network = keras.Model(input, x)


input_1 = keras.layers.Input((32, 32, 3))
input_2 = keras.layers.Input((32, 32, 3))

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = keras.layers.Lambda(euclidean_distance, output_shape=(1,))(
    [tower_1, tower_2]
)
normal_layer = keras.layers.BatchNormalization()(merge_layer)
output_layer = keras.layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = keras.Model(inputs=[input_1, input_2], outputs=output_layer)


# Compute Euclidean distance & create output layer
def euclidean_distance(vects):
    x, y = vects
    sum_square = ops.sum(ops.square(x - y), axis=1, keepdims=True)
    return ops.sqrt(ops.maximum(sum_square, keras.backend.epsilon()))



In [91]:
# [0.25 mark] TODO: Compile the model with  optimizer and loss function
# Build and compile the model
epochs = 5
batch_size = 16
margin = 1  # Margin for contrastive loss.
def loss(margin=1):
    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):


        square_pred = ops.square(y_pred)
        margin_square = ops.square(ops.maximum(margin - (y_pred), 0))
        return ops.mean((1 - y_true) * square_pred + (y_true) * margin_square)

    return contrastive_loss
siamese.compile(loss=loss(margin=margin), optimizer="adam", metrics=["accuracy"])
siamese.summary()

Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_30            │ (None, 32, 32, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_31            │ (None, 32, 32, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_20             │ (None, 10)             │          7,542 │ input_layer_30[0][0],  │
│ (Functional)              │                        │                │ input_layer_31[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_8 (Lambda)         │ (None, 1)              │              0 │ functional_20[0][0],   │
│                           │                        │                │ functional_20[1][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_27    │ (None, 1)              │              4 │ lambda_8[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_17 (Dense)          │ (None, 1)              │              2 │ batch_normalization_2… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 7,548 (29.48 KB)

 Trainable params: 6,740 (26.33 KB)

 Non-trainable params: 808 (3.16 KB)

In [92]:
x_train_1 = pairs_train[:, 0]  # x_train_1.shape is (60000, 28, 28)
x_train_2 = pairs_train[:, 1]

In [93]:
x_val_1 = pairs_val[:, 0]  # x_val_1.shape = (60000, 28, 28)
x_val_2 = pairs_val[:, 1]

In [ ]:
# [0.25 mark] TODO: Train the model for 5 epochs and evaluate it
# Train the model
history = siamese.fit(
    [x_train_1, x_train_2],
    labels_train,
    validation_data=([x_val_1, x_val_2], labels_val),
    batch_size=batch_size,
    epochs=epochs,
)


Epoch 1/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.5980 - loss: 0.2368 - val_accuracy: 0.6506 - val_loss: 0.2161
Epoch 2/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.6441 - loss: 0.2196 - val_accuracy: 0.6561 - val_loss: 0.2143
Epoch 3/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.6476 - loss: 0.2169 - val_accuracy: 0.6677 - val_loss: 0.2077
Epoch 4/5
1509/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6578 - loss: 0.2131

In [73]:
history.history


{'accuracy': [0.5051000118255615,
  0.6278749704360962,
  0.6514999866485596,
  0.659974992275238,
  0.6665999889373779],
 'loss': [0.25032278895378113,
  0.22655034065246582,
  0.21740593016147614,
  0.2135920226573944,
  0.2113029509782791],
 'val_accuracy': [0.5864250063896179,
  0.6641499996185303,
  0.6640999913215637,
  0.6743000149726868,
  0.6796749830245972],
 'val_loss': [0.23945613205432892,
  0.21057458221912384,
  0.20995672047138214,
  0.20659877359867096,
  0.20391547679901123]}

In [85]:
results = siamese.evaluate([testX, testY],labels_test)


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 10000, 10000
'y' sizes: 20000


## **Task 2 - Autoencoder for Image Reconstruction**  

Design and train an **Autoencoder** to reconstruct images from the **CIFAR-10** dataset.

### Instructions:

1. **Load and Preprocess MNIST:**
   - Load the **MNIST** dataset using `keras.datasets.MNIST.load_data()`.
   - Normalize the pixel values of images to the range `[0, 1]`.

2. **Build the Autoencoder Model:**
   - **Encoder:**
     - Input Layer: Take input of shape `(32, 32, 3)`.
     - Add **Conv2D** layers to reduce the spatial dimensions using max pooling.
     - Use **ReLU activation** for each layer.
   - **Latent Space:**
     - Add a Dense layer for the latent representation of the images.
   - **Decoder:**
     - Add **UpSampling2D** layers to reconstruct the original image size.
     - Use **sigmoid activation** in the final layer to output the reconstructed image.

3. **Train the Autoencoder:**
   - Use the appropriate optimizer and loss function.
   - Train for at least **10 epochs**


In [10]:
# Task 2: Autoencoder on MNIST

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np

# Load MNIST dataset
(X_train, _), (X_test, _) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [11]:
# [0.5 mark] Preprocess images
# Normalize data
x_train = X_train.astype('float32') / 255.
x_test = X_test.astype('float32') / 255.

# Reshape data (add dimension)
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

print(x_train.shape)

(60000, 28, 28, 1)


In [12]:
# Add Noise
noise_factor = 0.5
X_train_noise = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_test_noise = X_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)

X_train_noise = np.clip(X_train_noise, 0., 1.)
X_test_noise = np.clip(X_test_noise, 0., 1.)

In [43]:
# [1.5 mark] Build Autoencoder model

class Denoise(Model):
  def __init__(self):
    super(Denoise, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(28, 28, 1)),
      layers.Conv2D(16, (3, 3), activation='sigmoid', padding='same', strides=2),
      layers.Conv2D(8, (3, 3), activation='sigmoid', padding='same', strides=2)])

    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='sigmoid', padding='same'),
      layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='sigmoid', padding='same'),
      layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Denoise()

In [44]:
from tensorflow.keras import layers, losses


In [45]:
# [0.5 mark] TODO: Compile Train the model and validate on the test set + + early stopping
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())


In [46]:
callback = keras.callbacks.EarlyStopping(monitor='loss',
                                      patience=3)

In [58]:
model = autoencoder.fit(X_train_noise, x_train,
                epochs=10,
                shuffle=True,
                validation_data=(X_test_noise, x_test),
                callbacks=[callback]
                )

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.0077 - val_loss: 0.0075
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0076 - val_loss: 0.0075
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0076 - val_loss: 0.0077
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0075 - val_loss: 0.0075
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0075 - val_loss: 0.0074
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0075 - val_loss: 0.0073
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0074 - val_loss: 0.0073
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0074 - val_loss: 0.0073
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0074 - val_loss: 0.0072
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0073 - val_loss: 0.0073
